In [49]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification
import torch
from datasets import load_dataset
import pandas as pd
import os
import sys
from torchvision import transforms
from torch.utils.data import DataLoader
from pytorch_lightning import LightningModule
from torchmetrics.classification import MultilabelF1Score
from torchmetrics import AUROC
import torch
import onnx
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

In [26]:
notebook_dir = os.getcwd()
project_dir = os.path.dirname(notebook_dir)

# Add the src directory to sys.path
src_dir = os.path.join(project_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from utils import generate_class_weights, show_batch_images
from data import ChestXray14HFDataset
from trainers import MultiLabelLightningModule

## load dataframes

In [13]:
data_path = "/cluster/home/taheeraa/datasets/chestxray-14"

In [16]:
labels = [
    "Atelectasis", 
    "Cardiomegaly",
    "Effusion", 
    "Infiltration", 
    "Mass",
    "Nodule",
    "Pneumonia",
    "Pneumothorax",  
    "Consolidation",
    "Edema",
    "Emphysema",
    "Fibrosis",
    "Pleural_Thickening",
    "Hernia"
]
file_path_train = data_path + '/train_official.txt'
file_path_val = data_path + '/val_official.txt'
file_path_test = data_path + '/test_official.txt'

columns = ['Image Filename'] + labels

df_train = pd.read_csv(file_path_train, sep='\s+', names=columns)
df_val = pd.read_csv(file_path_val, sep='\s+', names=columns)
df_test = pd.read_csv(file_path_test, sep='\s+', names=columns)

# Finding all image paths, and mapping them to the DataFrame
subfolders = [f"images_{i:03}/images" for i in range(1, 13)]  # Generates 'images_001' to 'images_012'
path_mapping = {}
for subfolder in subfolders:
    full_folder_path = os.path.join(data_path, subfolder)
    for img_file in os.listdir(full_folder_path):
        path_mapping[img_file] = os.path.join(full_folder_path, img_file)

# Update the DataFrame using the mapping
df_train['Full Image Path'] = df_train['Image Filename'].map(path_mapping)
df_val['Full Image Path'] = df_val['Image Filename'].map(path_mapping)
df_test['Full Image Path'] = df_test['Image Filename'].map(path_mapping)

# Move 'Full Image Path' to the front of the DataFrame
cols_train = ['Full Image Path'] + [col for col in df_train.columns if col != 'Full Image Path']
cols_val = ['Full Image Path'] + [col for col in df_val.columns if col != 'Full Image Path']
cols_test = ['Full Image Path'] + [col for col in df_test.columns if col != 'Full Image Path']
df_train = df_train[cols_train]
df_val = df_val[cols_val]
df_test = df_test[cols_test]

# Drop 'Image Filename' column
train_df = df_train.drop(columns=['Image Filename'])
val_df = df_val.drop(columns=['Image Filename'])
test_df = df_test.drop(columns=['Image Filename'])

# Create class weights
df_train_calculate_weights = df_train.drop(columns=['Full Image Path']).to_numpy()
class_weights_dict = generate_class_weights(df_train_calculate_weights, multi_class=False, one_hot_encoded=True)
class_weights_list = [class_weights_dict[i] for i in class_weights_dict]
class_weights_tensor = torch.tensor(class_weights_list, dtype=torch.float32)

label_weights_dict = {labels[i]: class_weights_dict[i] for i in range(len(labels))}
print(f"Class weights: {label_weights_dict}")

print(f"Train dataframe shape: {df_train.shape} (1 size larger than expected due to 'Full Image Path')")
print(f"Train columns: {df_train.columns}")
print(f"Labels: {labels}")
print(f"Number of labels: {len(labels)}")


Class weights: {'Atelectasis': 0.06666666666666667, 'Cardiomegaly': 0.6968494101318529, 'Effusion': 3.3449700199866754, 'Infiltration': 0.6563995293502418, 'Mass': 0.41714855433698905, 'Nodule': 1.4048125349748182, 'Pneumonia': 1.2366502463054188, 'Pneumothorax': 6.52051948051948, 'Consolidation': 2.237433155080214, 'Edema': 2.024516129032258, 'Emphysema': 4.1494214876033055, 'Fibrosis': 4.01664, 'Pleural_Thickening': 4.63601108033241, 'Hernia': 2.6055007784120394}
Train dataframe shape: (75312, 16) (1 size larger than expected due to 'Full Image Path')
Train columns: Index(['Full Image Path', 'Image Filename', 'Atelectasis', 'Cardiomegaly',
       'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
       'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis',
       'Pleural_Thickening', 'Hernia'],
      dtype='object')
Labels: ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphys

## load dataset and transforms

In [22]:
img_size = 224
num_workers = 4
pin_memory = False
batch_size = 32

In [14]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR, antialias=True),
    transforms.CenterCrop(img_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=7),
    transforms.ToTensor(),
    normalize,
])

val_transforms = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR, antialias=True),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    normalize,
])

In [17]:
train_dataset = ChestXray14HFDataset(
    dataframe=train_df, transform=train_transforms)
val_dataset = ChestXray14HFDataset(
    dataframe=val_df, transform=val_transforms)
test_dataset = ChestXray14HFDataset(
    dataframe=test_df, transform=val_transforms)

In [23]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=pin_memory
)

## trainer

In [27]:

torch.backends.cudnn.benchmark = True


class MultiLabelLightningModule(LightningModule):
    def __init__(self, model, criterion, learning_rate, num_labels, scheduler, optimizer):
        super().__init__()
        self.model_arg = "swin"
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.num_labels = num_labels
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.log_step_interval = 100

        self.f1_score = MultilabelF1Score(
            num_labels=self.num_labels, threshold=0.5, average='macro')

        self.f1_score_micro = MultilabelF1Score(
            num_labels=self.num_labels, threshold=0.5, average='micro')
        
        self.auroc = AUROC(
            task="multilabel",
            num_labels=self.num_labels,
            average="macro",
        )

    def forward(self, pixel_values):
        return self.model(pixel_values)

    def step(self, batch):
        pixel_values = batch['pixel_values']
        labels = batch['labels']
        
        if self.model_arg == 'swin':
            logits = self.forward(pixel_values.logit)
        else:
            logits = self.forward(pixel_values)

        loss = self.criterion(logits, labels)
        return loss, logits, labels

    def training_step(self, batch, batch_idx):
        loss, logits, labels = self.step(batch)
        self.log('train_loss', loss, on_step=True,
                 on_epoch=True, prog_bar=True, logger=True)
        # Update metrics
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)

        # Log metrics
        if batch_idx % self.log_step_interval == 0:
            self.log('train_f1', f1, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_f1_micro', f1_micro, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_auroc', auroc, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('train_loss', loss, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
        
        return loss
        
    def validation_step(self, batch, batch_idx):
        loss, logits, labels = self.step(batch)
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)

        if batch_idx % self.log_step_interval == 0:
            self.log('val_f1', f1, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
            self.log('val_f1_micro', f1_micro, on_step=True,
                        on_epoch=True, prog_bar=True, logger=True)
            self.log('val_auroc', auroc, on_step=True,
                        on_epoch=True, prog_bar=True, logger=True)
            self.log('val_loss', loss, on_step=True,
                     on_epoch=True, prog_bar=True, logger=True)
        
    def test_step(self, batch, batch_idx):
        loss, logits, labels = self.step(batch)
        f1 = self.f1_with_sigmoid(logits, labels)
        f1_micro = self.f1_micro_with_sigmoid(logits, labels)
        auroc = self.auroc_with_sigmoid(logits, labels)

        self.log('test_loss', loss)
        self.log('test_f1', f1)
        self.log('test_f1_micro', f1_micro)
        self.log('test_auroc', auroc)

        return {'test_loss': loss, 'test_f1': f1, 'test_f1_micro': f1_micro}
    
    def configure_optimizers(self):
        return [self.optimizer], [self.scheduler]

    def f1_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.f1_score(preds, labels)

    def f1_micro_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.f1_score_micro(preds, labels)
    
    def auroc_with_sigmoid(self, logits, labels):
        preds = torch.sigmoid(logits)
        return self.auroc(preds, labels.type(torch.int32))

In [34]:
logger = TensorBoardLogger(save_dir="test")

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1, 
    verbose=True,
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',  
    min_delta=0.00,
    patience=3,
    verbose=True,
    mode='min'
)

## set up training

In [44]:
#(self, model, criterion, learning_rate, num_labels, scheduler, optimizer):
id2label = {id: label for id, label in enumerate(labels)}
label2id = {label: id for id, label in id2label.items()}
    
model = Swinv2ForImageClassification.from_pretrained(
    "microsoft/swinv2-tiny-patch4-window8-256",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-tiny-patch4-window8-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([14, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([14]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=3, bias=False)
                  )
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96

In [51]:
criterion = torch.nn.BCEWithLogitsLoss()
learning_rate = 0.0005
num_labels = len(labels)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = {'scheduler': scheduler, 'monitor': 'val_loss'}

num_epochs=20

In [52]:
training_module = MultiLabelLightningModule(
    model=model,
    criterion=criterion,
    learning_rate=learning_rate,
    num_labels=num_labels,
    scheduler=scheduler,
    optimizer=optimizer,
)

pl_trainer = Trainer(
    max_epochs=num_epochs,
    logger=logger,
    callbacks=[checkpoint_callback, early_stop_callback],
)


RuntimeError: You set `--ntasks=4` in your SLURM bash script, but this variable is not supported. HINT: Use `--ntasks-per-node=4` instead.

## start the training

In [ ]:
pl_trainer.fit(
    training_module,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

## test

In [ ]:
pl_trainer.test(
    model=training_module,
    dataloaders=test_loader,
)